## Chronic Kidney Disease diagnoses codes extraction
### SQL Query
The following query finds all diagnoses corresponding to stage 4 or 5 CKD, or end stage kidney disease, and returns the number of diagnoses for each.

```sql
SELECT
  d1.icd_code,
  COUNT(d2.icd_code) AS occurrence_count,
  d1.long_title
FROM
  `physionet-data.mimiciv_hosp.d_icd_diagnoses` AS d1
JOIN
  `physionet-data.mimiciv_hosp.diagnoses_icd` AS d2
ON
  d1.icd_code = d2.icd_code
WHERE
  (LOWER(d1.long_title) LIKE '%kidney disease%'
    AND ( LOWER(d1.long_title) LIKE '%stage 4%'
      OR LOWER(d1.long_title) LIKE '%stage 5%'
      OR LOWER(d1.long_title) LIKE '%stage iv%'
      OR LOWER(d1.long_title) LIKE '%stage v%'
      OR LOWER(d1.long_title) LIKE '%end stage renal disease%' )
    AND NOT ( LOWER(d1.long_title) LIKE '%unspecified%' ))
  OR ( LOWER(d1.long_title) LIKE '%end stage renal disease%' )
GROUP BY
  d1.icd_code,
  d1.long_title
ORDER BY
  occurrence_count DESC;

```

The results are stored in `ckd_codes.json`

The following query counts how often each pair of diagnoses (mentioning CKD at any stage) appears.
```sql
WITH kidney_disease_codes AS (
  SELECT icd_code, long_title
  FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`
  WHERE LOWER(long_title) LIKE '%kidney disease%'
)

SELECT
  LEAST(d1_titles.long_title, d2_titles.long_title) AS diagnosis_1,
  GREATEST(d1_titles.long_title, d2_titles.long_title) AS diagnosis_2,
  COUNT(*) AS pair_occurrence_count
FROM
  `physionet-data.mimiciv_hosp.diagnoses_icd` AS d1
JOIN
  kidney_disease_codes AS d1_titles ON d1.icd_code = d1_titles.icd_code
JOIN
  `physionet-data.mimiciv_hosp.diagnoses_icd` AS d2 ON d1.hadm_id = d2.hadm_id
JOIN
  kidney_disease_codes AS d2_titles ON d2.icd_code = d2_titles.icd_code
WHERE
  d1.icd_code != d2.icd_code  -- Ensure we are counting pairs of different diagnoses
GROUP BY
  diagnosis_1,
  diagnosis_2
ORDER BY
  pair_occurrence_count DESC;

```
We learn that when hypertensive CKD with "unspecified" CKD is mentioned, there is often another diagnosis specifying which specific stage of CKD it is.
However, "CKD, unspecified" is still the most prevalent diagnosis. Since it could be any stage, this diagnosis should be excluded. The corresponding ICD codes are 5859 and N189.


In [4]:
import pandas as pd
import json

In [5]:
with open('ckd_codes.json', 'r') as f:
    data = pd.DataFrame(json.load(f))

In [6]:
data

,icd_code,occurrence_count,long_title
0,5856,8885,End stage renal disease
1,40391,7981,"Hypertensive chronic kidney disease, unspecifi..."
2,N186,5131,End stage renal disease
3,I120,3153,Hypertensive chronic kidney disease with stage...
4,5854,2838,"Chronic kidney disease, Stage IV (severe)"
5,N184,2076,"Chronic kidney disease, stage 4 (severe)"
6,I132,1844,Hypertensive heart and chronic kidney disease ...
7,5855,682,"Chronic kidney disease, Stage V"
8,N185,585,"Chronic kidney disease, stage 5"
9,40311,279,"Hypertensive chronic kidney disease, benign, w..."


In [7]:
with open('ckd_codes.txt', 'w') as f:
    f.write(str(data.icd_code.tolist()))